In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

from tqdm import tqdm

In [2]:
!nvidia-smi

Mon Apr 14 18:39:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:01:00.0 Off |                   On |
| N/A   24C    P0             49W /  500W |      88MiB /  81920MiB |     N/A      Default |
|                                         |                        |            Disabled* |
+-----------------------------------------+-----

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = "cuda" if torch.cuda.is_available() else "cpu"

## Data Loading

In [4]:
# # LOCAL CLUSTER DIRECTORY FILE LOADING
# import pickle

# # Function to unpickle the dataset files
# def unpickle(file):
#     with open(file, 'rb') as fo:
#         data_dict = pickle.load(fo, encoding='bytes')
#     return data_dict

# # Function to load all data batches
# def load_data(batch_files):
#     all_data = []
#     all_labels = []
#     for batch_file in batch_files:
#         batch_file = DATASET_PATH + batch_file
#         batch = unpickle(batch_file)
#         all_data.append(batch[b'data'])  # Image data
#         all_labels.append(batch[b'labels'])  # Labels
#     data = np.concatenate(all_data)
#     labels = np.concatenate(all_labels)
#     return data, labels

# # Function to reshape image data
# def reshape_images(data):
#     # Reshape from (10000, 3072) to (10000, 32, 32, 3)
#     data = data.reshape(-1, 3, 32, 32)  # (10000, 3, 32, 32)
#     data = data.transpose(0, 2, 3, 1)  # (10000, 32, 32, 3)
#     return data

# # Load metadata (label names)
# def load_meta(file):
#     file = DATASET_PATH + file
#     meta = unpickle(file)
#     label_names = [name.decode('utf-8') for name in meta[b'label_names']]  # Decode bytes to strings
#     return label_names

# # Example usage
# # Files for the training batches and metadata
# batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
# DATASET_PATH = "/datasets/CS747/cifar-10-batches-py/"
# test_batch_file = 'test_batch'
# meta_file = 'batches.meta'

# # Load training data
# data, labels = load_data(batch_files)
# data = reshape_images(data)  # Reshape image data into (10000, 32, 32, 3)

# # Load test data
# test_data, test_labels = load_data([test_batch_file])
# test_data = reshape_images(test_data)  # Reshape test data into (10000, 32, 32, 3)

# # Load label names (metadata)
# label_names = load_meta(meta_file)

# # Example of accessing one image and its label:
# image_index = 3
# image = data[image_index]  # 32x32x3 image
# label = labels[image_index]
# label_name = label_names[label]

# print(f"Image shape: {image.shape}, Label: {label}, Label name: {label_name}")

In [5]:
# setup data loader
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])


# LOAD FROM TORCHVISION

train_mnist = datasets.MNIST(root='./data', download=True, train=True, transform=transforms.ToTensor())
test_mnist = datasets.MNIST(root='./data', download=True, train=False, transform=transforms.ToTensor())


#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
full_train_cifar = datasets.CIFAR10(root='./data', download=True, train=True, transform=transform_train)
test_cifar = datasets.CIFAR10(root='./data', download=True, train=False, transform=transform_test)

In [6]:
BATCH_SIZE = 128

In [7]:
# Calculate sizes for split (e.g., 90% training, 10% validation)
num_train = int(0.9 * len(full_train_cifar))  # 45,000 images
num_val = len(full_train_cifar) - num_train    # 5,000 images

train_dataset, val_dataset = random_split(full_train_cifar, [num_train, num_val])

train_loader_cifar = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader_cifar = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE*2, shuffle=False)
test_loader_cifar = DataLoader(dataset=test_cifar, batch_size=BATCH_SIZE*2, shuffle=False)

train_loader_mnist = DataLoader(dataset=train_mnist, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
# MNIST CNN CONFIGS

mnist_cnn_config = {
    "learning_rate" : 0.01, # learning rate
    "weight_decay" : 3.5e-5,
    }

cifar_config = {
    "learning_rate" : 0.01,
    "weight_decay" : 3.5e-5,
    "epochs": 100
}



In [9]:
from collections import OrderedDict
import torch.nn as nn


class SmallCNN(nn.Module):
    def __init__(self, drop=0.5):
        super(SmallCNN, self).__init__()

        self.num_channels = 1
        self.num_labels = 10

        activ = nn.ReLU(True)

        self.feature_extractor = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(self.num_channels, 32, 3)),
            ('relu1', activ),
            ('conv2', nn.Conv2d(32, 32, 3)),
            ('relu2', activ),
            ('maxpool1', nn.MaxPool2d(2, 2)),
            ('conv3', nn.Conv2d(32, 64, 3)),
            ('relu3', activ),
            ('conv4', nn.Conv2d(64, 64, 3)),
            ('relu4', activ),
            ('maxpool2', nn.MaxPool2d(2, 2)),
        ]))

        self.classifier = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(64 * 4 * 4, 200)),
            ('relu1', activ),
            ('drop', nn.Dropout(drop)),
            ('fc2', nn.Linear(200, 200)),
            ('relu2', activ),
            ('fc3', nn.Linear(200, self.num_labels)),
        ]))

        for m in self.modules():
            if isinstance(m, (nn.Conv2d)):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        nn.init.constant_(self.classifier.fc3.weight, 0)
        nn.init.constant_(self.classifier.fc3.bias, 0)

    def forward(self, input):
        features = self.feature_extractor(input)
        #print(features.shape)
        logits = self.classifier(features.view(-1, 64 * 4 * 4))
        return logits

In [10]:
## Resnet Model

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.residual = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = F.relu(self.bn2(self.conv2(out)), inplace=True)
        out = out + self.residual(x)
        out = F.relu(out, inplace=True)
        return out


class ResNet(nn.Module):
    def __init__(self, block: BasicBlock, num_blocks: list[int], num_classes: int=10):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer0 = self.make_layer(block, 64, num_blocks[0], stride=1)
        self.layer1 = self.make_layer(block, 128, num_blocks[1], stride=2)
        self.layer2 = self.make_layer(block, 256, num_blocks[2], stride=2)
        self.layer3 = self.make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)


    
    def make_layer(self, block, out_channels, num_blocks, stride):
        layers = []

        layers.append(block(self.in_channels, out_channels, stride=stride))
        self.in_channels = out_channels * block.expansion

        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)), inplace=True)
        out = self.layer0(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        logits = self.linear(out)
        return logits

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


In [11]:
#model_test = ResNet18().to(device)

In [12]:
# x_rand = torch.randn((1, 3, 32, 32)).cuda()
# print(x_rand.shape)
# output = model_test(x_rand)
# print(output.shape)

In [13]:
def train_one_epoch(model,  dataloader, criterion, optimizer, device, max_grad_norm=None):
    model.train()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for images, labels in tqdm(dataloader, desc="Training", leave=False):

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        
        if max_grad_norm:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs.detach(), -1)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += images.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc = 'Validation', leave=False):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)

            preds = torch.argmax(outputs, -1)
            running_corrects += torch.sum((preds == labels)).item()
            total_samples += labels.size(0)
        
        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples

        return epoch_loss, epoch_acc

# def adjust_learning_rate(optimizer, epoch, config):
#     """decrease the learning rate"""
#     lr = config["learning_rate"]

#     if epoch >= 100:
#         lr = config["learning_rate"] * 0.001
#     elif epoch >= 90:
#         lr = config["learning_rate"] * 0.01
#     elif epoch >= 75:
#         lr = config["learning_rate"] * 0.1

#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

In [14]:
def save_checkpoint(model, optimizer, epoch, val_loss, checkpoint_dir, top_checkpoints, max_checkpoints=5):
    """
    Save a checkpoint if it's among the top best ones.
    
    Args:
        model (torch.nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer state to save.
        epoch (int): The current epoch.
        val_loss (float): The validation loss metric used for ranking.
        checkpoint_dir (str): Directory to save the checkpoints.
        top_checkpoints (list): List of tuples (val_loss, filename) for current top checkpoints.
        max_checkpoints (int, optional): Maximum number of checkpoints to save. Defaults to 5.
    """
    # Ensure the checkpoint directory exists.
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Construct a filename that contains the epoch and validation loss.
    checkpoint_filename = os.path.join(checkpoint_dir, f"model_epoch_{epoch+1}_valloss_{val_loss:.4f}.pth")
    
    # Create the checkpoint dictionary
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': val_loss,
    }
    
    # Save checkpoint to disk
    torch.save(checkpoint, checkpoint_filename)
    
    # Add the checkpoint to your top checkpoints list (we assume lower loss is better)
    top_checkpoints.append((val_loss, checkpoint_filename))
    
    # Sort the checkpoints by validation loss (ascending order)
    top_checkpoints.sort(key=lambda x: x[0])
    
    # If more than max_checkpoints, delete the worst checkpoint (last one in sorted list)
    if len(top_checkpoints) > max_checkpoints:
        worst_checkpoint = top_checkpoints.pop()  # pop the highest loss checkpoint
        if os.path.exists(worst_checkpoint[1]):
            os.remove(worst_checkpoint[1])
            print(f"Removed checkpoint: {worst_checkpoint[1]}")

In [15]:
def train_model(model, train_loader, val_loader, config, scheduler=None, device="cuda"):
    num_epochs = config["epochs"]
    
    top_checkpoints = []
    checkpoint_dir = config.get("checkpoint_dir", "./checkpoints")

    history = {
        "train_loss" : [],
        "train_acc" : [],
        "val_loss" : [],
        "val_acc" : []
    }

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), 
                    lr = config["learning_rate"], 
                    momentum= 0.9,
                    weight_decay=config["weight_decay"])

    scheduler = MultiStepLR(optimizer=optimizer, milestones= [75, 90, 100], gamma=0.1)
    
    for epoch in range(1, num_epochs+1):
        #print(f"Epoch {epoch}/{num_epochs}")

        #adjust_learning_rate(optimizer, epoch, config=config)

        train_loss, train_acc = train_one_epoch(
            model = model, dataloader = train_loader, 
            criterion=criterion, optimizer=optimizer, 
            device=device, max_grad_norm=1.0
        )

        val_loss, val_acc = validate(model=model, dataloader=val_loader, criterion=criterion, device=device)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch [{epoch}/{num_epochs}] | "
              f"LR: {scheduler.get_last_lr()[0]:.3f} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")

        if scheduler:
            scheduler.step()

        save_checkpoint(model, optimizer, epoch, val_loss, checkpoint_dir, top_checkpoints, max_checkpoints=5)
        
    

    return model, history, top_checkpoints

In [16]:
def main():
    model = ResNet18().to(device)

    model, history = train_model(model, 
                                 train_loader=train_loader_cifar, 
                                 val_loader=val_loader_cifar,
                                 config= cifar_config
    )

    print("Training Done")
    print('================================================================')

    test_loss, test_acc = validate(model, dataloader=test_loader_cifar, criterion=nn.CrossEntropyLoss(), device=device)
    print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

    return history


In [65]:
if __name__ == '__main__':
    history = main()

Training:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch [1/100] | LR: 0.010 | Train Loss: 1.5923 | Train Acc: 41.05% | Val Loss: 1.3970 | Val Acc: 47.84%


Epoch [2/100] | LR: 0.010 | Train Loss: 1.1824 | Train Acc: 57.12% | Val Loss: 1.5524 | Val Acc: 49.64%


Epoch [3/100] | LR: 0.010 | Train Loss: 0.9979 | Train Acc: 64.37% | Val Loss: 1.1023 | Val Acc: 60.52%


Epoch [4/100] | LR: 0.010 | Train Loss: 0.8744 | Train Acc: 68.76% | Val Loss: 0.9641 | Val Acc: 65.74%


...

Epoch [97/100] | LR: 0.000 | Train Loss: 0.0030 | Train Acc: 99.95% | Val Loss: 0.3978 | Val Acc: 91.40%
Removed checkpoint: ./checkpoints/model_epoch_98_valloss_0.3978.pth


Epoch [98/100] | LR: 0.000 | Train Loss: 0.0031 | Train Acc: 99.94% | Val Loss: 0.4304 | Val Acc: 90.88%
Removed checkpoint: ./checkpoints/model_epoch_99_valloss_0.4304.pth


Epoch [99/100] | LR: 0.000 | Train Loss: 0.0028 | Train Acc: 99.95% | Val Loss: 0.4165 | Val Acc: 90.82%
Removed checkpoint: ./checkpoints/model_epoch_100_valloss_0.4165.pth


Epoch [100/100] | LR: 0.000 | Train Loss: 0.0027 | Train Acc: 99.95% | Val Loss: 0.4339 | Val Acc: 90.74%
Removed checkpoint: ./checkpoints/model_epoch_101_valloss_0.4339.pth


ValueError: too many values to unpack (expected 2)

In [ ]:
# _, test_acc = validate(model, dataloader=test_loader_cifar, criterion=nn.CrossEntropyLoss(), device=device)
# print(f"Test accuracy: {test_acc}")

## Adver training

In [ ]:
def create_adv_samples(model, inputs, labels, config, random_start=True):
    epsilon = config["epsilon"]
    step_size = config["step_size"]
    num_steps = config["num_steps"]
    #norm = config["norm"]

    model.eval()
    # Keep original clean samples
    x_clean = inputs.clone().detach()


    if random_start:
        delta = torch.rand_like(inputs, device=device) * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(inputs, device=device)
    
    delta = torch.clamp(delta, -epsilon, epsilon)
    delta.requires_grad = True

    for step in range(num_steps):
        x_adv = x_clean + delta
        
        # for stable batch norm stats and to disable dropout
        with torch.enable_grad():
            outputs = model(x_adv)
        loss = F.cross_entropy(outputs, labels)

        grad = torch.autograd.grad(loss, delta)[0]
        delta = delta.detach() + step_size * torch.sign(grad)
        delta = torch.clamp(delta, -epsilon, epsilon)

        # clamp adversarial sample to valid pixel range        
        x_adv_clamped = torch.clamp(x_clean + delta, 0.0, 1.0)
        
        # get accurate delat from clamped x
        delta = x_adv_clamped - x_clean
        delta = delta.detach()
        
        if step < num_steps - 1:
            delta.requires_grad = True
    
    # final adversarial sample
    x_adv_batch = (x_clean + delta).detach()

    return x_adv_batch


def mart_loss(logits_clean, logits_adv, labels, lambda_reg):
    # BCE LOSS = standard cross entropy + margin maximization
    # get the probabilty distribution for the adversarial logits
    probs_adv = F.softmax(logits_adv, dim=-1)

    # sort it and get the two highest values/prediction
    tmp1 = torch.argsort(input=probs_adv, dim=-1)[-2:]
    
    # get the max probability for the incorrect class prediction. basically if the highest probability is for the correct class, get the next higest probability
    labels_new = torch.where(tmp1[:,-1] == labels, input=tmp1[:,-2], other=tmp1[:,-1])

    bce_loss = F.cross_entropy(logits_adv, labels) + F.nll_loss(torch.log(1 - probs_adv + 1e-12), labels_new)


    # KL term
    kl = nn.KLDivLoss(reduction='none')
    probs_clean = F.softmax(logits_clean, dim=-1)
    probs_true = torch.gather(probs_clean, dim=1, index=(labels.unsqueeze(1)).long()).squeeze()
    reg_loss = torch.sum(torch.sum(kl(torch.log(probs_adv + 1e-12), probs_clean), dim=1) * 1.000001 - probs_true) / labels.size[0]

    loss = bce_loss + float(lambda_reg) * reg_loss

    return loss

    # reg_loss = torch.sum(torch.sum(kl(torch.log(probs_adv + 1e-12), probs_nat), dim=1) * (1 - true_probs)) / inputs.size(0)

    # loss_robust = (1.0 / batch_size) * torch.sum(torch.sum(kl(torch.log(adv_probs + 1e-12), nat_probs), dim=1) * (1.0000001 - true_probs))
    pass



def adv_train_one_epoch(model,  dataloader, criterion, optimizer, config, device="cuda", max_grad_norm=None):
    #model.train()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for images, labels in tqdm(dataloader, desc="Training", leave=False):

        images = images.to(device)
        labels = labels.to(device)


        model.train()
        x_adv = create_adv_samples(model, inputs=images, labels=labels, config=config, random_start=True)

        optimizer.zero_grad()

        # Forward pass
        logits_clean = model(images)
        logits_adv = model(x_adv)

        loss = mart_loss(logits_clean, logits_adv, labels, lambda_reg=config["lambda_reg"])
        loss.backward()
        
        if max_grad_norm:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs.detach(), -1)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += images.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    return epoch_loss, epoch_acc